# Libraries

In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import h2o
from h2o.automl import H2OAutoML
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_columns', None)

# Data

## Intake data

### Data url

In [3]:
intake_url = "https://data.austintexas.gov/resource/wter-evkm.csv?$limit=500000"

### Data pull

In [4]:
df_intakes = pd.read_csv(intake_url)

### Data preview

In [ ]:
df_intakes.head()

## Outcome data

### Data url

In [6]:
outcome_url = "https://data.austintexas.gov/resource/9t4d-g238.csv?$limit=500000"

### Data pull

In [7]:
df_outcomes = pd.read_csv(outcome_url)

### Data preview

In [ ]:
df_outcomes.head(20)

# EDA

## Missing values in intakes

In [ ]:
# Check for missing values in df_outcomes
missing_values = df_intakes.isnull().sum()
print("Missing values in df_intakes:\n", missing_values)

## Missing values in outcomes

In [ ]:
# Check for missing values in df_outcomes
missing_values = df_outcomes.isnull().sum()
print("Missing values in df_outcomes:\n", missing_values)

## Unique values in color

In [ ]:
df_intakes['color'].nunique()

In [ ]:
df_outcomes['color'].nunique()

High cardinality exists here

## Unique values in found_location

In [ ]:
df_intakes['found_location'].nunique()

High cardinality exists here

## Unique values in breed

In [ ]:
df_intakes['breed'].nunique()

In [ ]:
df_outcomes['breed'].nunique()

High cardinality here too

# Preprocessing

## Fix datetime column

### Intakes

In [16]:
df_intakes['datetime'] = pd.to_datetime(df_intakes['datetime'])

In [17]:
df_intakes['monthyear'] = pd.to_datetime(df_intakes['datetime2']).dt.strftime('%m-%Y')

In [18]:
df_intakes = df_intakes.drop(columns=['datetime2'])

### Outcomes

In [19]:
df_outcomes['date_of_birth'] = pd.to_datetime(df_outcomes['date_of_birth'])
df_outcomes['monthyear'] = pd.to_datetime(df_outcomes['monthyear'], format='%m-%Y')

In [20]:
df_outcomes['datetime'] = pd.to_datetime(df_outcomes['datetime'], format='mixed', utc=True)

## Dropping null values from both dataframes

For **df_intakes**, missing names can be handled later in a binary variable, while, variables like *sex_upon_intake* will need to be omitted from the analysis.

For **df_outcomes**, missing values in age and sex will be omitted. 

In [21]:
df_intakes = df_intakes.dropna(subset=['sex_upon_intake'])

In [22]:
df_outcomes = df_outcomes.dropna(subset=['outcome_type', 'sex_upon_outcome', 'age_upon_outcome'])

## Combining both dataframes into one

### First, sort by date

In [23]:
df_intakes_sorted = df_intakes.sort_values('datetime')
df_outcomes_sorted = df_outcomes.sort_values('datetime')

### Combine both dataframes into one

In [24]:
combined_df = pd.merge(df_intakes_sorted, df_outcomes_sorted, on='animal_id', suffixes=('_intake', '_outcome'), how='inner')

## Age variable fix

In [25]:
def convert_age_to_days(age_str, name_str=None):
    """Convert age string to numeric days"""
    if pd.isna(age_str):
        return None
    
    age_str = age_str.lower()
    
    # Handle "0 years" case - check if newborn based on name containing "grams"
    if age_str.strip() == "0 years":
        if name_str and "grams" in str(name_str).lower():
            return 0  # Newborn
        else:
            return None  # Missing age data
    
    total_days = 0
    
    # Find all number-unit pairs
    patterns = [
        (r'(\d+)\s*year', 365),
        (r'(\d+)\s*month', 30),
        (r'(\d+)\s*week', 7),
        (r'(\d+)\s*day', 1)
    ]
    
    for pattern, multiplier in patterns:
        matches = re.findall(pattern, age_str)
        for match in matches:
            total_days += int(match) * multiplier
    
    return total_days if total_days > 0 else None

def convert_age_to_years(age_str, name_str=None):
    """Convert age string to numeric years (decimal)"""
    days = convert_age_to_days(age_str, name_str)
    return days / 365 if days is not None else None

In [26]:
df_intakes['age_upon_intake_days'] = df_intakes.apply(lambda row: convert_age_to_days(row['age_upon_intake'], row.get('name')), axis=1)
df_intakes['age_upon_intake_years'] = df_intakes.apply(lambda row: convert_age_to_years(row['age_upon_intake'], row.get('name')), axis=1)

df_outcomes['age_upon_outcome_days'] = df_outcomes.apply(lambda row: convert_age_to_days(row['age_upon_outcome'], row.get('name')), axis=1)
df_outcomes['age_upon_outcome_years'] = df_outcomes.apply(lambda row: convert_age_to_years(row['age_upon_outcome'], row.get('name')), axis=1)


In [ ]:
print("Sample conversions:")
test_ages = ["7 years", "2 weeks", "5 months", "10 days"]
for age in test_ages:
    print(f"{age} -> {convert_age_to_days(age)} days, {convert_age_to_years(age):.2f} years")

In [28]:
combined_df['age_upon_outcome_days'] = combined_df.apply(lambda row: convert_age_to_days(row['age_upon_outcome'], row.get('name')), axis=1)
combined_df['age_upon_outcome_years'] = combined_df.apply(lambda row: convert_age_to_years(row['age_upon_outcome'], row.get('name')), axis=1)

combined_df['age_upon_intake_days'] = combined_df.apply(lambda row: convert_age_to_days(row['age_upon_intake'], row.get('name')), axis=1)
combined_df['age_upon_intake_years'] = combined_df.apply(lambda row: convert_age_to_years(row['age_upon_intake'], row.get('name')), axis=1)

In [ ]:
combined_df.isna().sum()

In [ ]:
df_outcomes.isna().sum()

In [ ]:
df_intakes.isnull().sum()

### Dropping null values for age upon intake & age upon outcome

In [32]:
df_intakes = df_intakes.dropna(subset=['age_upon_intake_days'])

In [33]:
df_outcomes = df_outcomes.dropna(subset=['age_upon_outcome_days'])

### Dropping age upon intake and age upon outcome columns

In [34]:
df_intakes = df_intakes.drop(columns=['age_upon_intake'])
df_outcomes = df_outcomes.drop(columns=['age_upon_outcome'])

In [ ]:
# Null values in 

# 1. Check for null values in the dataset
print(df_intakes.isnull().sum())

# 2. Check for null values in the dataset
print(df_outcomes.isnull().sum())

## Breed variable fix

In [36]:
def engineer_breed_features(df, breed_column='breed'):
    """Extract meaningful features from breed information"""
    
    # Create a copy to avoid modifying original
    df = df.copy()
    
    # Animal type already exists, so skipping that classification
    
    # Mixed breed indicators
    df['is_mixed'] = df[breed_column].str.contains('Mix|/', case=False, na=False)
    df['num_breeds'] = df[breed_column].str.count('/') + 1
    df['num_breeds'] = df['num_breeds'].where(df['is_mixed'], 1)
    
    # Size categories (based on common breed patterns)
    toy_breeds = ['Chihuahua', 'Yorkshire', 'Toy', 'Maltese', 'Pomeranian', 'Papillon', 'Miniature']
    small_breeds = ['Terrier', 'Beagle', 'Cocker', 'Dachshund', 'Corgi', 'Pug', 'Shih Tzu']
    large_breeds = ['Retriever', 'Shepherd', 'Mastiff', 'Great Dane', 'Rottweiler', 'Great Pyrenees', 'Newfoundland']
    
    def categorize_size(breed_str):
        if pd.isna(breed_str):
            return 'Unknown'
        breed_str = str(breed_str)
        if any(toy in breed_str for toy in toy_breeds):
            return 'Toy'
        elif any(large in breed_str for large in large_breeds):
            return 'Large'
        elif any(small in breed_str for small in small_breeds):
            return 'Small'
        else:
            return 'Medium'
    
    df['size_category'] = df[breed_column].apply(categorize_size)
    
    # Working/sport groups
    working_breeds = ['Shepherd', 'Cattle Dog', 'Border Collie', 'Australian Kelpie', 'Husky', 'Malamute']
    sporting_breeds = ['Retriever', 'Pointer', 'Spaniel', 'Setter', 'Vizsla']
    terrier_breeds = ['Terrier']
    
    df['is_working'] = df[breed_column].str.contains('|'.join(working_breeds), case=False, na=False)
    df['is_sporting'] = df[breed_column].str.contains('|'.join(sporting_breeds), case=False, na=False)
    df['is_terrier'] = df[breed_column].str.contains('Terrier', case=False, na=False)
    
    # Popular breed indicators
    popular_dog_breeds = ['Labrador', 'Golden Retriever', 'Pit Bull', 'Chihuahua', 'Beagle', 'German Shepherd']
    popular_cat_breeds = ['Domestic Shorthair', 'Domestic Longhair', 'Siamese']
    
    df['is_popular_dog'] = df[breed_column].str.contains('|'.join(popular_dog_breeds), case=False, na=False)
    df['is_popular_cat'] = df[breed_column].str.contains('|'.join(popular_cat_breeds), case=False, na=False)
    
    # Exotic/unusual animals
    exotic_animals = ['Snake', 'Lizard', 'Turtle', 'Bird', 'Rabbit', 'Ferret', 'Pig', 'Goat', 'Chicken']
    df['is_exotic'] = df[breed_column].str.contains('|'.join(exotic_animals), case=False, na=False)
    
    # Primary breed (for mixed breeds, take the first one)
    def extract_primary_breed(breed_str):
        if pd.isna(breed_str):
            return None
        breed_str = str(breed_str)
        if '/' in breed_str:
            return breed_str.split('/')[0].strip()
        elif 'Mix' in breed_str:
            return breed_str.replace(' Mix', '').strip()
        else:
            return breed_str.strip()
    
    df['primary_breed'] = df[breed_column].apply(extract_primary_breed)
    
    # Breed complexity score (more complex = more mixed)
    df['breed_complexity'] = df['num_breeds'] + df['is_mixed'].astype(int)
    
    return df

In [37]:
combined_df = engineer_breed_features(combined_df, 'breed_outcome')

In [ ]:
combined_df.info()

In [39]:
combined_df = combined_df.drop(columns=['breed_intake','breed_outcome'])

## Has name column

### Fix cases with weight in place of name

Some newborn animals have their weight put into the name column. Let's fix this

In [40]:
# Replace weight entries with null values
combined_df['name_intake'] = combined_df['name_intake'].replace(r'^\d+\s*grams?$', pd.NA, regex=True)
combined_df['name_outcome'] = combined_df['name_outcome'].replace(r'^\d+\s*grams?$', pd.NA, regex=True)

### If name at outcome is not null, has_name will be 1, 0 otherwise

In [41]:
combined_df['has_name'] = combined_df['name_outcome'].notnull().astype(int)

## Keeping only Dogs and Cats

Less than 6% if instances are not dogs or cats. Thus, for this model, dropping those instances makes sense.

In [42]:
combined_df = combined_df[combined_df['animal_type_outcome'].isin(['Dog', 'Cat'])]

## Fix found_location column to extract feature

The good thing here is that the location an animal is found is noted down, but, due to high cardinality, this feature also needs to be engineered. Capturing the jurisdiction, and area related information is going to be engineered.

In [43]:
def engineer_location_features(df, location_column='found_location'):
    """Extract meaningful features from found_location"""
    
    df = df.copy()
    
    # Extract jurisdiction/city from parentheses or end of string
    def extract_jurisdiction(location_str):
        if pd.isna(location_str):
            return None
        location_str = str(location_str).strip()
        
        # Pattern for (TX) or (County)
        paren_match = re.search(r'\(([^)]+)\)', location_str)
        if paren_match:
            return paren_match.group(1)
        
        # For cases like "Manor(TX)" without space
        no_space_match = re.search(r'([A-Za-z]+)\([^)]+\)$', location_str)
        if no_space_match:
            return no_space_match.group(1)
        
        return None
    
    # Extract city/area name
    def extract_city(location_str):
        if pd.isna(location_str):
            return None
        location_str = str(location_str).strip()
        
        # Handle "Outside jurisdiction"
        if 'outside jurisdiction' in location_str.lower():
            return 'Outside Jurisdiction'
        
        # Extract city before " in " or before parentheses
        if ' in ' in location_str:
            parts = location_str.split(' in ')
            if len(parts) > 1:
                city_part = parts[1].split('(')[0].strip()
                return city_part
        
        # For direct formats like "Austin (TX)" or "Manor(TX)"
        city_match = re.search(r'^([A-Za-z\s]+)(?:\s*\(|$)', location_str)
        if city_match:
            return city_match.group(1).strip()
        
        return location_str
    
    # Apply extractions
    df['jurisdiction'] = df[location_column].apply(extract_jurisdiction)
    df['city_area'] = df[location_column].apply(extract_city)
    
    # Austin area classifications
    austin_areas = ['Austin', 'Travis', 'Manor', 'Pflugerville', 'Cedar Park', 'Round Rock', 
                   'Lakeway', 'Bee Cave', 'West Lake Hills', 'Rollingwood', 'Sunset Valley']
    
    surrounding_areas = ['Williamson', 'Hays', 'Bastrop', 'Caldwell', 'Georgetown', 
                        'Leander', 'Cedar Creek', 'Elgin', 'Dripping Springs']
    
    df['is_austin_metro'] = df['city_area'].isin(austin_areas)
    df['is_surrounding_area'] = df['city_area'].isin(surrounding_areas)
    df['is_outside_jurisdiction'] = df['city_area'] == 'Outside Jurisdiction'
    
    # Core Austin vs suburbs
    df['is_core_austin'] = df['city_area'] == 'Austin'
    df['is_travis_county'] = df['city_area'].isin(['Austin', 'Travis', 'Manor'])
    
    # Distance categories (approximate)
    def categorize_distance(city):
        if pd.isna(city):
            return 'Unknown'
        if city in ['Austin', 'Travis']:
            return 'Core'
        elif city in ['Manor', 'Pflugerville', 'West Lake Hills', 'Rollingwood', 'Sunset Valley']:
            return 'Close Suburbs'
        elif city in ['Cedar Park', 'Round Rock', 'Lakeway', 'Bee Cave']:
            return 'Far Suburbs'
        elif city == 'Outside Jurisdiction':
            return 'Outside'
        else:
            return 'Other'
    
    df['distance_category'] = df['city_area'].apply(categorize_distance)
    
    # Has specific address (contains street info)
    # df['has_address_detail'] = df[location_column].str.contains(r'\d+\s+\w+', na=False)
    
    return df

In [44]:
combined_df = engineer_location_features(combined_df, 'found_location')

In [45]:
def print_location_summary(df):
    """Print summary of location features"""
    print("Location Feature Summary:")
    print(f"Core Austin: {df['is_core_austin'].sum()}")
    print(f"Travis County: {df['is_travis_county'].sum()}")
    print(f"Austin Metro: {df['is_austin_metro'].sum()}")
    print(f"Outside Jurisdiction: {df['is_outside_jurisdiction'].sum()}")
    print("\nDistance categories:")
    print(df['distance_category'].value_counts())

In [ ]:
print_location_summary(combined_df)

## Duplicate case fix

Duplicates for animal IDs exist because some animals keep returning, and, there are features here to capture these intricacies.

In [47]:
# Return visit feature engineering for combined dataframe
combined_df = combined_df.sort_values(['animal_id', 'datetime_intake'])

# Visit count for each animal
combined_df['visit_count'] = combined_df.groupby('animal_id').cumcount() + 1

# Return visit indicators
combined_df['is_return_visit'] = (combined_df['visit_count'] > 1).astype(int)
combined_df['is_frequent_returner'] = (combined_df['visit_count'] > 2).astype(int)

# Days since last visit
combined_df['days_since_last_visit'] = combined_df.groupby('animal_id')['datetime_intake'].diff().dt.days
combined_df['days_since_last_visit'] = combined_df['days_since_last_visit'].fillna(-1)

# Previous outcome type
combined_df['previous_outcome_type'] = combined_df.groupby('animal_id')['outcome_type'].shift(1)
combined_df['previous_outcome_type'] = combined_df['previous_outcome_type'].fillna('First Visit')

In [ ]:
combined_df.info()

## High cardinality check

Some columns have high cardinalities, thus, depending on their importance, they will either be trasnformed, or dropped.

These columns are:

1. name_intake: drop
2. name_outcome: drop
3. color_intake: drop
4. color_outcome: drop
5. primary_breed: reduce/drop
6. monthyear_intake: split
7. age_upon_outcome: drop
8. age_upon_intake: drop
9. outcome_type: reduce to adoption or no adoption later
10. found_location: drop
11. city_area: reduce
12. outcome_subtype: drop
13. animal_id: drop before analysis
14. monthyear_outcome: split
15. jurisdiction: reduce

In [ ]:
cat_cols = combined_df.select_dtypes(include=['object', 'category']).columns
cat_unique_counts = combined_df[cat_cols].nunique()
print(cat_unique_counts.sort_values(ascending=False))

In [50]:
# 1. Drop columns
columns_to_drop = ['name_intake', 'name_outcome', 'color_intake', 'color_outcome', 
                   'age_upon_outcome', 'age_upon_intake', 'found_location', 
                   'outcome_subtype', 'animal_id']
combined_df = combined_df.drop(columns=columns_to_drop)

# 2. Reduce primary_breed to top 10
top_breeds = combined_df['primary_breed'].value_counts().head(10).index
combined_df['primary_breed'] = combined_df['primary_breed'].apply(
    lambda x: x if x in top_breeds else 'Other'
)

# 3. Extract month from monthyear_intake
combined_df['month_intake'] = pd.to_datetime(combined_df['monthyear_intake']).dt.month
combined_df = combined_df.drop(columns=['monthyear_intake'])

# 4. Extract month from monthyear_outcome  
combined_df['month_outcome'] = pd.to_datetime(combined_df['monthyear_outcome']).dt.month
combined_df = combined_df.drop(columns=['monthyear_outcome'])

# 5. Reduce city_area to top 10
top_cities = combined_df['city_area'].value_counts().head(10).index
combined_df['city_area'] = combined_df['city_area'].apply(
    lambda x: x if x in top_cities else 'Other'
)

# 6. Reduce jurisdiction to top 10
top_jurisdictions = combined_df['jurisdiction'].value_counts().head(10).index
combined_df['jurisdiction'] = combined_df['jurisdiction'].apply(
    lambda x: x if x in top_jurisdictions else 'Other'
)

## Create target variable **Is_Adopted**

In [ ]:
combined_df['outcome_type'].value_counts()

There will be a good class balance doing this, something the Long Beach struggled with. Albeit, this is a simpler analysis.

In [52]:
combined_df['Is_adopted'] = (combined_df['outcome_type'] == 'Adoption').astype(int)

In [53]:
combined_df = combined_df.drop(columns=['outcome_type'])

## Deconstructing time related features

Since datetime data types don't give enough interpretable information to the ML algorithm, understanding cyclical and temporal patterns would be better to identify with datetime transformations.

In [ ]:
combined_df.info()

### Extract Basic Time Components

In [55]:
# For datetime_intake
combined_df['intake_year'] = combined_df['datetime_intake'].dt.year
combined_df['intake_day'] = combined_df['datetime_intake'].dt.day
combined_df['intake_dayofweek'] = combined_df['datetime_intake'].dt.dayofweek

# For date_of_birth
combined_df['birth_year'] = combined_df['date_of_birth'].dt.year
combined_df['birth_month'] = combined_df['date_of_birth'].dt.month

### Seasonal Features

In [56]:
# Season (1=Winter, 2=Spring, 3=Summer, 4=Fall)
combined_df['intake_season'] = combined_df['month_intake'].map({
    12: 1, 1: 1, 2: 1,    # Winter
    3: 2, 4: 2, 5: 2,     # Spring
    6: 3, 7: 3, 8: 3,     # Summer
    9: 4, 10: 4, 11: 4    # Fall
})

# Is weekend
combined_df['intake_is_weekend'] = (combined_df['intake_dayofweek'] >= 5).astype(int)

In [ ]:
combined_df.info()

## Removing all other outcome related information

Steps taken to prevent data leakage. Some other repetitive columns will also be dropped here.

In [58]:
combined_df = combined_df.drop(columns=['age_upon_outcome_days', 'age_upon_outcome_years', 'age_upon_intake_years', 'month_outcome', 'datetime_outcome', 'animal_type_outcome', 'sex_upon_outcome', 'datetime_intake'])

In [ ]:
combined_df.info()

## Drop null values of *age_upon_intake_days*

In [60]:
combined_df = combined_df.dropna(subset=['age_upon_intake_days'])

In [ ]:
combined_df.info()

# ML Model

## Correlation matrix

In [ ]:
# Select only numerical columns
numerical_cols = combined_df.select_dtypes(include=[np.number]).columns

# Create correlation matrix
correlation_matrix = combined_df[numerical_cols].corr()

# Create the correlation plot
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, 
            annot=True, 
            cmap='coolwarm', 
            center=0,
            square=True,
            fmt='.2f',
            cbar_kws={'shrink': 0.8})

plt.title('Correlation Matrix of Numerical Variables', fontsize=16, pad=20)
plt.tight_layout()
plt.show()

In [ ]:
combined_df.info()

## H20 automl run 1

In [ ]:
# H2O AutoML Modeling

# Initialize H2O
h2o.init()

"""## Data Preparation for H2O"""

# Create a copy for modeling
model_data = combined_df.copy()

# Convert categorical columns to strings for H2O
categorical_cols = ['animal_type_intake', 'sex_upon_intake', 'intake_type', 'intake_condition',
                   'size_category', 'distance_category', 'primary_breed', 'city_area', 
                   'jurisdiction', 'previous_outcome_type']

for col in categorical_cols:
    if col in model_data.columns:
        model_data[col] = model_data[col].astype(str)

# Convert to H2O Frame
h2o_data = h2o.H2OFrame(model_data)

# Set categorical columns
for col in categorical_cols:
    if col in h2o_data.columns:
        h2o_data[col] = h2o_data[col].asfactor()

# Set target as factor for classification
h2o_data['Is_adopted'] = h2o_data['Is_adopted'].asfactor()

"""## Train-Test Split"""

train, test = h2o_data.split_frame(ratios=[0.8], seed=42)

# Define features and target
target = 'Is_adopted'
features = [col for col in h2o_data.columns if col != target]

print(f"Training set: {train.nrows} rows")
print(f"Test set: {test.nrows} rows")
print(f"Features: {len(features)}")

"""## H2O AutoML with Specified Algorithms"""

# Define the algorithms we want to include (using correct H2O names)
include_algos = ["XGBoost", "GBM", "DRF", "StackedEnsemble"]

# Run AutoML
aml = H2OAutoML(
    max_models=50,  # Increase for more hyperparameter combinations
    max_runtime_secs=14400,  # 1 hour max runtime
    include_algos=include_algos,
    seed=42,
    balance_classes=True,  # Handle class imbalance
    sort_metric="AUC"
)

# Train the models
aml.train(x=features, y=target, training_frame=train, validation_frame=test)



In [ ]:
"""## Model Performance Evaluation"""

# View the leaderboard
lb = aml.leaderboard
print("AutoML Leaderboard:")
print(lb.head(rows=10))

# Get the best model
best_model = aml.leader


In [ ]:
# Performance on test set
test_performance = best_model.model_performance(test)
print("\nBest Model Performance on Test Set:")
print(f"AUC: {test_performance.auc():.4f}")
print(f"Accuracy: {test_performance.accuracy()[0][0]:.4f}")
print(f"Precision: {test_performance.precision()[0][0]:.4f}")
print(f"Recall: {test_performance.recall()[0][0]:.4f}")
print(f"F1 Score: {test_performance.F1()[0][0]:.4f}")

# Confusion Matrix
cm = test_performance.confusion_matrix()
print("\nConfusion Matrix:")
print(cm)

In [ ]:
h2o.cluster().shutdown()

In [ ]:
# Get confusion matrix at 0.5 threshold
cm_05 = test_performance.confusion_matrix(thresholds=[0.5])
print("Confusion Matrix at 0.5 threshold:")
print(cm_05)

# Get metrics at 0.5 threshold
metrics_05 = test_performance.metric('f1', thresholds=[0.5])
print(f"F1 at 0.5 threshold: {metrics_05[0][1]:.4f}")

## Best model performance metrics

In [ ]:
# Performance metrics at 0.5 threshold
cm_05 = test_performance.confusion_matrix(thresholds=[0.5])
print("Confusion Matrix at 0.5 threshold:")
print(cm_05)

# Extract key metrics at 0.5 threshold
f1_05 = test_performance.metric('f1', thresholds=[0.5])[0][1]
precision_05 = test_performance.metric('precision', thresholds=[0.5])[0][1] 
recall_05 = test_performance.metric('recall', thresholds=[0.5])[0][1]
accuracy_05 = test_performance.metric('accuracy', thresholds=[0.5])[0][1]

print(f"\nMetrics at 0.5 threshold:")
print(f"Accuracy: {accuracy_05:.4f}")
print(f"Precision: {precision_05:.4f}")
print(f"Recall: {recall_05:.4f}")
print(f"F1 Score: {f1_05:.4f}")

## Saving model hyperparameters

In [ ]:
# 1. Get best model
best_model = aml.leader
print(f"Best Model: {best_model.model_id}")
print(f"Algorithm: {best_model.algo}")

# 2. Get hyperparameters
print("\nHyperparameters:")
params = best_model.params
for key, value in params.items():
   if value['actual'] is not None:
       print(f"{key}: {value['actual']}")

# For ensemble, show base models
if "StackedEnsemble" in best_model.model_id:
   print(f"\nBase models:")
   for model in best_model.params['base_models']['actual']:
       print(f"- {model}")

# 3. Save model (H2O format - pickle won't work for H2O models)
model_path = h2o.save_model(best_model, path="./", force=True)
print(f"\nModel saved to: {model_path}")



In [ ]:
# For FastAPI, you'll need to convert to MOJO or use H2O server
# Save as MOJO (better for production)
mojo_path = best_model.download_mojo(path="./", get_genmodel_jar=True)
print(f"MOJO saved to: {mojo_path}")

In [ ]:
h2o.cluster().shutdown()

In [ ]:
x.info()

In [ ]:
combined_df.info()